In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def plot_correlation_raincloud():
    # Set up the data
    metabolites = ['Omega-3 fatty acids', 'Glucose', 'Albumin signal area']
    outcomes = ['Intima-media thickness', 'Changes in appetite', 'Changes in sleep pattern']

    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create the raincloud plot
    for i, outcome in enumerate(outcomes):
        for j, metabolite in enumerate(metabolites):
            r = correlations[i, j]
            p = p_values[i, j]
            if not np.isnan(r) and not np.isnan(p):
                color = 'red' if r > 0 else 'blue'
                size = -np.log10(p) * 100  # Adjust size based on p-value
                
                # Add jitter to y-position for better visualization
                y_jitter = np.random.normal(0, 0.05)
                
                # Plot the point
                ax.scatter(j, i + y_jitter, c=color, s=size, alpha=0.7)
                
                # Add annotation for all correlations
                ax.annotate(f'r={r:.4f}\np={p:.4f}', (j, i + y_jitter), 
                            xytext=(5, 5), textcoords='offset points', 
                            fontsize=8, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

#     # Add violin plots for distribution (only for non-nan values)
#     for j, metabolite in enumerate(metabolites):
#         valid_correlations = correlations[:, j][~np.isnan(correlations[:, j])]
#         if len(valid_correlations) > 0:
#             sns.violinplot(x=[j]*len(valid_correlations), y=valid_correlations, ax=ax, 
#                            inner=None, color="lightgray", alpha=0.5)

    ax.set_yticks(range(len(outcomes)))
    ax.set_yticklabels(outcomes)
    ax.set_xticks(range(len(metabolites)))
    ax.set_xticklabels(metabolites, rotation=0, ha='right')
  #  ax.set_xlabel("Metabolites")
   #ax.set_ylabel("Outcomes")

    # Add a color bar legend
    sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=-0.15, vmax=0.15))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("Partial Coefficient")

    plt.tight_layout()
    plt.show()

# Call the function
plot_correlation_raincloud()


In [ ]:
import pandas as pd
import glob
import numpy as np

In [ ]:
import numpy as np
import pandas as pd

def clean_NMR(df_NMR, threshold=5):
    # Removing the first column (likely an index or unnecessary column)
    df_NMR = df_NMR.iloc[:, 1:]

    # Extract and keep the 'eid' column
    eid_col = df_NMR.iloc[:, 0]
    df_NMR = df_NMR.iloc[:, 1:]

    # Remove columns related to visits 2, 3, and 4
    words_to_remove = ['Visit2', 'Visit3', 'Visit4']
    columns_to_keep = ~df_NMR.columns.str.contains('|'.join(words_to_remove))
    df_NMR = df_NMR.loc[:, columns_to_keep]

    # Drop the specific 'Date_of_attending_assessment_centre_Visit1_0' column
    df_NMR = df_NMR.drop(columns=['Date_of_attending_assessment_centre_Visit1_0'])

    print(df_NMR.shape)
    print("Missing values per column before imputation:", df_NMR.isna().sum())

    # Mean imputation for missing values
    df_NMR = df_NMR.fillna(df_NMR.mean())
    print("Missing values per column after imputation:", df_NMR.isna().sum())

    # Filter out values greater than threshold * standard deviation
    df_NMR_filt = df_NMR.copy()
    for col in df_NMR.columns:
        mean = df_NMR[col].mean()
        std = df_NMR[col].std()
        df_NMR_filt.loc[abs(df_NMR[col] - mean) > threshold * std, col] = np.nan
    
    # Mean imputation again for new NaN values after filtering
    df_NMR_filt = df_NMR_filt.fillna(df_NMR_filt.mean())
    print("Missing values per column after filtering and imputation:", df_NMR_filt.isna().sum())

    # Add 1 to all values
    df_NMR_filt = df_NMR_filt.add(1, axis='index')

    # Log transformation
    df_NMR_filt = df_NMR_filt.apply(lambda x: np.log(x))

    # Normalize the data
    df_NMR_filt = df_NMR_filt.apply(lambda x: (x - x.mean()) / x.std())

    # Remove '_Visit1_0' from column names
    df_NMR_filt.columns = df_NMR_filt.columns.str.replace('_Visit1_0', '')

    # Add back the 'eid' column
    df_NMR_filt = pd.concat([eid_col, df_NMR_filt], axis=1)

    # Save the cleaned DataFrame to a CSV file
    df_NMR_filt.to_csv('NMR_cleaned.csv', index=False)
    
    return df_NMR_filt


In [ ]:

# Clean NMR data
df_NMR = pd.read_csv('NMR_metabolic_biomarkers_with_dates.csv', delimiter=',')
df_NMR_cleaned = clean_NMR(df_NMR)
# about 3.5 min to run

In [ ]:
df_NMR_cleaned.columns.to_list()

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
columns_to_select = ['eid', 'Glucose', 'Omega-3_Fatty_Acids', 'Citrate', 'Creatinine', 'Albumin']
selected_df = df_NMR_cleaned[columns_to_select]
# or
selected_df = df_NMR_cleaned.loc[:, columns_to_select]
selected_df

In [ ]:
df_combined = pd.read_csv('df_combined.csv')
selected_df2 = df_combined.rename(columns={'ID': 'eid'})

In [ ]:
import pandas as pd

# Assuming df_combined is your DataFrame
columns_to_select = [
    'eid',
    'Age.At.MHQ.1',
    'Gender_x',
    'Body mass index (BMI)',
    'Smoking status',
    'Moderate.Physical.Activity_x',
    'Diabetes_x', 
    'Ever thought that life not worth living',
    'Ever had prolonged loss of interest in normal activities',
    'Recent lack of interest or pleasure in doing things',
    'Recent poor appetite or overeating',
    'Trouble.sleeping',
    'Sleeping.change',
    'Trouble falling or staying asleep, or sleeping too much',
    'Sleep duration',
    'Sleeplessness / insomnia'
]


selected_df2_ = selected_df2.loc[:, columns_to_select]


selected_df2_

In [ ]:
combined_data = pd.merge(selected_df2_, selected_df, on='eid', how='inner')
combined_data.columns

In [ ]:
df_imputed = combined_data.fillna(combined_data.mean())

In [ ]:
import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Return summary statistics
    return pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia'))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating'))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Trouble.sleeping'))

# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table


In [ ]:
import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_var=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_var (str, optional): The name of the covariate variable column. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariate if provided
    if covariate_var:
        X[covariate_var] = df[covariate_var]

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [covariate_var] if covariate_var else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })

    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', 'Age.At.MHQ.1'))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Trouble.sleeping', 'Age.At.MHQ.1'))

# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table


In [ ]:
import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_var=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_var (str, optional): The name of the covariate variable column. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariate if provided
    if covariate_var:
        X[covariate_var] = df[covariate_var]

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [covariate_var] if covariate_var else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })

    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', 'Gender_x'))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Trouble.sleeping', 'Gender_x'))

# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table


In [ ]:
import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_vars=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_vars (list of str, optional): The names of the covariate variable columns. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariates if provided
    if covariate_vars:
        X = pd.concat([X] + [df[cov] for cov in covariate_vars], axis=1)

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    
    
      # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [", ".join(covariate_vars)] if covariate_vars else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })


    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', ['Age.At.MHQ.1', 'Gender_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Ever thought that life not worth living', ['Age.At.MHQ.1', 'Gender_x']))




# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table

In [ ]:
#smoking status

import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_vars=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_vars (list of str, optional): The names of the covariate variable columns. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariates if provided
    if covariate_vars:
        X = pd.concat([X] + [df[cov] for cov in covariate_vars], axis=1)

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    
    
      # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [", ".join(covariate_vars)] if covariate_vars else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })


    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Ever thought that life not worth living', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status']))




# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table

In [ ]:




import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_vars=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_vars (list of str, optional): The names of the covariate variable columns. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariates if provided
    if covariate_vars:
        X = pd.concat([X] + [df[cov] for cov in covariate_vars], axis=1)

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    
    
      # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [", ".join(covariate_vars)] if covariate_vars else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })


    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Ever thought that life not worth living', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x']))




# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table

In [ ]:


import pandas as pd
import statsmodels.api as sm

def perform_ols_regression(df, dependent_var, independent_var, covariate_vars=None):
    """
    Perform OLS regression and return a summary table with dependent variable, 
    independent variable, coefficient, and p-value.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    dependent_var (str): The name of the dependent variable column.
    independent_var (str): The name of the independent variable column.
    covariate_vars (list of str, optional): The names of the covariate variable columns. Default is None.

    Returns:
    DataFrame: A DataFrame containing the regression summary.
    """
    # Define the dependent variable
    y = df[dependent_var]

    # Define the independent variable
    X = df[[independent_var]]

    # Add covariates if provided
    if covariate_vars:
        X = pd.concat([X] + [df[cov] for cov in covariate_vars], axis=1)

    # Add a constant to the independent variables (for the intercept term)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    
    
      # Extract the coefficient and p-value
    coefficient = model.params[independent_var]
    p_value = model.pvalues[independent_var]

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Dependent Variable': [dependent_var],
        'Independent Variable': [independent_var],
        'Covariate': [", ".join(covariate_vars)] if covariate_vars else ['None'],
        'Coefficient': [coefficient],
        'P-value': [p_value],
        'P-value < 0.05': [p_value < 0.05]  # Highlight if p-value is less than 0.05
    })


    return summary_df

# Example usage:
summary_tables = []
summary_tables.append(perform_ols_regression(df_imputed, 'Albumin', 'Trouble.sleeping', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Trouble falling or staying asleep, or sleeping too much', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Glucose', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Recent lack of interest or pleasure in doing things', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeping.change', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Creatinine', 'Sleeplessness / insomnia', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Omega-3_Fatty_Acids', 'Recent poor appetite or overeating', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))
summary_tables.append(perform_ols_regression(df_imputed, 'Citrate', 'Ever thought that life not worth living', ['Age.At.MHQ.1', 'Gender_x', 'Smoking status', 'Moderate.Physical.Activity_x', 'Diabetes_x']))




# Concatenate all summary tables
combined_summary_table = pd.concat(summary_tables, ignore_index=True)

# Apply styling
def highlight_p_value(s):
    if s['P-value'] < 0.05:
        return ['background-color: lightgreen' for _ in s]
    return ['' for _ in s]

combined_summary_table = combined_summary_table.style.apply(highlight_p_value, axis=1)

# Display the styled combined summary table
combined_summary_table